# Get information from zip


In [1]:
from mllib.utils.dvc import load_params
from pathlib import Path

### Get dict of labeld pathes in processed zip


In [2]:

%load_ext autoreload
%autoreload 2
from src.utils.files import get_label_dict

params = load_params()
train_zip = params['data']['train_zip']
targets_file = 'train/targets.tsv'
labels_dict = get_label_dict(zip_path=train_zip, targets_file=targets_file, prefix='train_processed/', postfix='.png')
labels_dict

{'train_processed/5d1f7e43366513a1d0a6ec5640c3dc24.png': 1,
 'train_processed/9a701a4536a05b6610a590a9fe702ed8.png': 1,
 'train_processed/cad0b8547008d1524c1a0e5fd51f9908.png': 1,
 'train_processed/4bbe607e7dc95460e2cc1a6ee5f4dfa6.png': 0,
 'train_processed/30fb32cba90b34af26f3f14f5d636805.png': 0,
 'train_processed/fa33445afe71a6dc18e4881c053da5be.png': 0,
 'train_processed/2f1a399df5f41cf3730253d07d7756ae.png': 0,
 'train_processed/cfb6bb7aa22ae864c2f3e08047b2cd3a.png': 0,
 'train_processed/8de1d8568032a2c5fa8301313b5487bc.png': 1,
 'train_processed/072aec7c0d3ed87387a310fd7c88f9a4.png': 0,
 'train_processed/9ac84ceb21b973a97a42a0a2871da2a4.png': 1,
 'train_processed/5c8ad64dd55b682d0e7e10e57d3bdaff.png': 1,
 'train_processed/11622347865947d971e527e3d971f2f9.png': 1,
 'train_processed/3ac8078b6e2ca534ddc1062d94f7cfcf.png': 0,
 'train_processed/4d261acb44afb98432f66578aea1bf1f.png': 0,
 'train_processed/37961a381e9df505afdc1bd2c3483e1c.png': 0,
 'train_processed/08a0aedcf23f974821c455

### Split train_dataset check label proportion

In [3]:
from src.utils.files import split_train_val_test
from collections import Counter

train_files, val_files, test_files = split_train_val_test(
    files_list=list(labels_dict.keys()), fraction=[0.6, 0.2, 0.2], seed=42
)

print(len(train_files), len(val_files), len(test_files))


def get_label_count(files: list, label_dict: dict, norm=True):
    c = Counter([label_dict[file] for file in files])
    print(c)
    if norm:
        c_total = c.total()
        for key, val in c.items():
            c[key] /= c_total

    return c


get_label_count(labels_dict.keys(), label_dict=labels_dict), get_label_count(
    train_files, label_dict=labels_dict
), get_label_count(val_files, label_dict=labels_dict), get_label_count(
    test_files, label_dict=labels_dict
)

8362 2787 2787
Counter({0: 7620, 1: 6316})
Counter({0: 4571, 1: 3791})
Counter({0: 1520, 1: 1267})
Counter({0: 1529, 1: 1258})


(Counter({0: 0.5467853042479908, 1: 0.4532146957520092}),
 Counter({0: 0.5466395599138962, 1: 0.4533604400861038}),
 Counter({0: 0.545389307499103, 1: 0.45461069250089703}),
 Counter({0: 0.5486185862935056, 1: 0.4513814137064944}))

In [17]:
import zipfile
from src.utils.files import extract_files_list

from mllib.utils.io import make_folder

train_processed_zip = params["data"]["processed_train_zip"]
train_processed_dir = Path(params["data"]["processed_train_dir"])

labels = list(set(labels_dict.values()))
make_folder(train_processed_dir)
for split, dir_name in zip(
    [train_files, val_files, test_files], ["train", "validate", "test"]
):
    for label in labels:
        filter_split = [file for file in split if labels_dict[file] == label]
        output = train_processed_dir / dir_name / str(label)
        make_folder(output)
        extract_files_list(
            zip_path=train_processed_zip, files=filter_split, output=output
        )

In [20]:
# check the number of files. Unpacced number of files is correct
import os

cpt = sum([len(files) for r, d, files in os.walk(train_processed_dir)])
cpt, len(list(labels_dict.keys()))

(13936, 13936)

In [21]:
# Simple extract

processed_dir = params["data"]["processed_dir"]

In [23]:
import shutil

train_processed_zip = Path(params["data"]["processed_train_zip"])
processed_dir = Path(params["data"]["processed_dir"])

shutil.unpack_archive(filename=train_processed_zip, extract_dir=processed_dir)

ReadError: data\processed\train_processed.zip is not a zip file